In [61]:
import glob
import pandas as pd
import time

In [62]:
#Already scrapped the weather folder which has many csv files
t1 = time.time()
files = glob.glob('weather/*.csv')
columns = ['station', 'date', 'tmpf', 'relh', 'sped', 'mslp',
           'p01i', 'vsby', 'gust_mph', 'skyc1', 'skyc2', 'skyc3']

# init empty DataFrame, like you might for a list
weather = pd.DataFrame(columns=columns)

for fp in files:
    city = pd.read_csv(fp)
    weather = weather.append(city)
t2 = time.time()
print(t2-t1)

67.29335260391235


In [4]:
t3 = time.time()
files = glob.glob('weather/*.csv')
weather_dfs = [pd.read_csv(fp) for fp in files]
weather = pd.concat(weather_dfs)
t4 = time.time()
print(t4-t3)

18.893413066864014


In [63]:
weather.head()

,station,date,tmpf,relh,sped,mslp,p01i,vsby,gust_mph,skyc1,skyc2,skyc3
0,PAAK,2014-01-01 00:16:00,35.6,80.51,16.1,NaN,0.0,5.0,21.9,SCT,BKN,OVC
1,PAAK,2014-01-01 00:36:00,37.4,74.98,15.0,NaN,0.0,10.0,23.0,SCT,SCT,BKN
2,PAAK,2014-01-01 00:56:00,37.4,74.98,15.0,NaN,0.0,10.0,NaN,FEW,SCT,BKN
3,PAAK,2014-01-01 01:16:00,37.4,74.98,16.1,NaN,0.0,10.0,26.5,FEW,SCT,BKN
4,PAAK,2014-01-01 01:36:00,37.4,74.98,19.6,NaN,0.0,8.0,27.6,SCT,BKN,OVC


In [64]:
weather = weather.set_index('date')

In [65]:
weather.loc[weather['station']=='DSM'].head()

,station,tmpf,relh,sped,mslp,p01i,vsby,gust_mph,skyc1,skyc2,skyc3
date,,,,,,,,,,,
2014-01-01 00:54:00,DSM,10.94,72.79,10.4,1024.9,0.0,10.0,NaN,FEW,M,M
2014-01-01 01:54:00,DSM,10.94,72.79,11.5,1025.4,0.0,10.0,NaN,OVC,M,M
2014-01-01 02:54:00,DSM,10.94,72.79,8.1,1025.3,0.0,10.0,NaN,BKN,M,M
2014-01-01 03:54:00,DSM,10.94,72.79,9.2,1025.3,0.0,10.0,NaN,OVC,M,M
2014-01-01 04:54:00,DSM,10.04,72.69,9.2,1024.7,0.0,10.0,NaN,BKN,M,M


In [70]:
weather = (weather.assign(date = lambda x: pd.to_datetime(x.index))
                 .set_index(['date']))

In [96]:
import numpy as np
a = pd.DataFrame({'a':[0,1,2]},index = np.array([1485707130, 1485707142, 1485708254]).astype('datetime64[s]'))
offset = pd.DateOffset(hours = 5, minutes = 30)
a.index = a.index + offset
a

,a
2017-01-29 21:55:30,0
2017-01-29 21:55:42,1
2017-01-29 22:14:14,2


In [28]:
temp = weather['tmpf']

C = (temp - 32)*5/9
C.head()

date
2014-01-01 00:16:00    2.0
2014-01-01 00:36:00    3.0
2014-01-01 00:56:00    3.0
2014-01-01 01:16:00    3.0
2014-01-01 01:36:00    3.0
Name: tmpf, dtype: float64

In [29]:
temp2 = weather.reset_index()[['station', 'date', 'tmpf']]
temp2['tmpf'] = (temp2['tmpf'] - 32) * 5 / 9
temp2.head()

,station,date,tmpf
0,PAAK,2014-01-01 00:16:00,2.0
1,PAAK,2014-01-01 00:36:00,3.0
2,PAAK,2014-01-01 00:56:00,3.0
3,PAAK,2014-01-01 01:16:00,3.0
4,PAAK,2014-01-01 01:36:00,3.0


In [37]:
weather = weather.reset_index()
weather.head()

,date,station,tmpf,relh,sped,mslp,p01i,vsby,gust_mph,skyc1,skyc2,skyc3
0,2014-01-01 00:16:00,PAAK,35.6,80.51,16.1,NaN,0.0,5.0,21.9,SCT,BKN,OVC
1,2014-01-01 00:36:00,PAAK,37.4,74.98,15.0,NaN,0.0,10.0,23.0,SCT,SCT,BKN
2,2014-01-01 00:56:00,PAAK,37.4,74.98,15.0,NaN,0.0,10.0,NaN,FEW,SCT,BKN
3,2014-01-01 01:16:00,PAAK,37.4,74.98,16.1,NaN,0.0,10.0,26.5,FEW,SCT,BKN
4,2014-01-01 01:36:00,PAAK,37.4,74.98,19.6,NaN,0.0,8.0,27.6,SCT,BKN,OVC


In [38]:
weather = weather.set_index(['station','date']).sort_index()

In [56]:
dsm = weather.loc['DSM']
dsm.index = pd.to_datetime(dsm.index)
hourly = dsm.resample('H').mean()
temp = hourly['tmpf'].sample(frac =0.5, random_state =1).sort_index()
sped = hourly['sped'].sample(frac = 0.5, random_state =2).sort_index()
print(temp.head())
print(sped.head())

date
2014-01-01 00:00:00    10.94
2014-01-01 02:00:00    10.94
2014-01-01 03:00:00    10.94
2014-01-01 04:00:00    10.04
2014-01-01 05:00:00    10.04
Name: tmpf, dtype: float64
date
2014-01-01 01:00:00    11.5
2014-01-01 02:00:00     8.1
2014-01-01 03:00:00     9.2
2014-01-01 04:00:00     9.2
2014-01-01 05:00:00    10.4
Name: sped, dtype: float64


In [57]:
#Even though indexes of both sped and temp are different div operator aligns and put Nan in indexes where it dont align
sped/temp

date
2014-01-01 00:00:00         NaN
2014-01-01 01:00:00         NaN
2014-01-01 02:00:00    0.740402
2014-01-01 03:00:00    0.840951
2014-01-01 04:00:00    0.916335
2014-01-01 05:00:00    1.035857
2014-01-01 06:00:00         NaN
2014-01-01 07:00:00         NaN
2014-01-01 08:00:00         NaN
2014-01-01 09:00:00         NaN
2014-01-01 10:00:00         NaN
2014-01-01 11:00:00         NaN
2014-01-01 12:00:00         NaN
2014-01-01 13:00:00    1.542969
2014-01-01 14:00:00         NaN
2014-01-01 15:00:00         NaN
2014-01-01 16:00:00    1.452581
2014-01-01 17:00:00         NaN
2014-01-01 18:00:00    1.556992
2014-01-01 19:00:00         NaN
2014-01-01 20:00:00         NaN
2014-01-01 22:00:00         NaN
2014-01-01 23:00:00    2.146402
2014-01-02 00:00:00         NaN
2014-01-02 01:00:00         NaN
2014-01-02 04:00:00         NaN
2014-01-02 05:00:00         NaN
2014-01-02 06:00:00         NaN
2014-01-02 07:00:00         NaN
2014-01-02 08:00:00         NaN
                         ...   
201

In [58]:
#division using div operator after filling missing values
sped.div(temp, fill_value=1)

date
2014-01-01 00:00:00     0.091408
2014-01-01 01:00:00    11.500000
2014-01-01 02:00:00     0.740402
2014-01-01 03:00:00     0.840951
2014-01-01 04:00:00     0.916335
2014-01-01 05:00:00     1.035857
2014-01-01 06:00:00     0.093721
2014-01-01 07:00:00    12.700000
2014-01-01 08:00:00     0.083195
2014-01-01 09:00:00     0.091408
2014-01-01 10:00:00    15.000000
2014-01-01 11:00:00     0.099602
2014-01-01 12:00:00    12.675000
2014-01-01 13:00:00     1.542969
2014-01-01 14:00:00    10.925000
2014-01-01 15:00:00    12.666667
2014-01-01 16:00:00     1.452581
2014-01-01 17:00:00     0.124069
2014-01-01 18:00:00     1.556992
2014-01-01 19:00:00     0.108342
2014-01-01 20:00:00    13.550000
2014-01-01 22:00:00    13.800000
2014-01-01 23:00:00     2.146402
2014-01-02 00:00:00    12.700000
2014-01-02 01:00:00    10.950000
2014-01-02 04:00:00     9.200000
2014-01-02 05:00:00     0.164474
2014-01-02 06:00:00     9.200000
2014-01-02 07:00:00     0.164474
2014-01-02 08:00:00    10.400000
     